In [1]:
import glob
import pandas as pd
import numpy as np


In [4]:
# import label

label = pd.read_csv('/workspace/brail_poc3/notebooks/Chen/Implement/label.csv', index_col=0)
label

,ID,Age_1,TAPA,CVLT,TMT
0,sub-032311,20,-0.235381,-0.371043,-0.423398
1,sub-032309,20,0.103865,0.160211,-0.296576
2,sub-032382,25,-0.224514,0.182326,-0.380918
3,sub-032302,20,-0.060740,-0.298945,-0.368657
4,sub-032315,20,-0.350309,0.130786,-0.354255
...,...,...,...,...,...
223,sub-032304,25,-0.096677,0.786817,0.948453
224,sub-032439,20,-0.250214,-0.287718,-0.356117
225,sub-032440,25,-0.375662,-0.536070,-0.323686
226,sub-032441,25,-0.101989,0.555243,-0.048483


In [5]:
label['ID'].to_list()

['sub-032311',
 'sub-032309',
 'sub-032382',
 'sub-032302',
 'sub-032315',
 'sub-032314',
 'sub-032316',
 'sub-032383',
 'sub-032307',
 'sub-032310',
 'sub-032312',
 'sub-032317',
 'sub-032404',
 'sub-032318',
 'sub-032319',
 'sub-032386',
 'sub-032320',
 'sub-032321',
 'sub-032322',
 'sub-032394',
 'sub-032324',
 'sub-032323',
 'sub-032528',
 'sub-032325',
 'sub-032313',
 'sub-032405',
 'sub-032384',
 'sub-032407',
 'sub-032327',
 'sub-032414',
 'sub-032418',
 'sub-032326',
 'sub-032415',
 'sub-032332',
 'sub-032331',
 'sub-032305',
 'sub-032308',
 'sub-032428',
 'sub-032333',
 'sub-032430',
 'sub-032301',
 'sub-032303',
 'sub-032336',
 'sub-032329',
 'sub-032431',
 'sub-032335',
 'sub-032337',
 'sub-032339',
 'sub-032306',
 'sub-032338',
 'sub-032340',
 'sub-032341',
 'sub-032442',
 'sub-032347',
 'sub-032444',
 'sub-032346',
 'sub-032350',
 'sub-032351',
 'sub-032348',
 'sub-032343',
 'sub-032342',
 'sub-032334',
 'sub-032356',
 'sub-032358',
 'sub-032446',
 'sub-032421',
 'sub-0323

In [6]:
# import feature

cohList = glob.glob('/workspace/brail_poc3/experiments/FC_Result_231103/coh/*')
pliList = glob.glob('/workspace/brail_poc3/experiments/FC_Result_231103/pli/*')
plvList = glob.glob('/workspace/brail_poc3/experiments/FC_Result_231103/plv/*')

excludeList = ['sub-032336', 'sub-032415', 'sub-032342', 'sub-032339']

FC = []
for i, j, k in zip(cohList,pliList,plvList):
    if i.split("/")[-1].split(".")[0] in label['ID'].to_list() and i.split("/")[-1].split(".")[0] not in excludeList:
        Sub = {}
        Sub["ID"] = i.split("/")[-1].split(".")[0]
        Sub["coh"] = np.load(i, allow_pickle='TRUE')
        Sub["pli"] = np.load(j, allow_pickle='TRUE')
        Sub["plv"] = np.load(k, allow_pickle='TRUE')
        Sub["Feature"] = np.array([Sub["coh"], Sub["pli"], Sub["plv"]])
        FC.append(Sub)
        
# FC_1 = [item for item in FC if item['cog'] == 1]
# FC_2 = [item for item in FC if item['cog'] == 2]

# sub-032336
# sub-032415
# sub-032342
# sub-032339

IDList = []
for i in FC:
    IDList.append(i['ID'])
# [item for item in IDList if item in excludeList]

In [42]:
feature = []
labelID = []
for i in FC:
    feature.append(i['Feature'])
    labelID.append(i['ID'])
feature = np.array(feature)
feature = feature.reshape(feature.shape[0], feature.shape[2], feature.shape[3], feature.shape[1])

# Check shape
print(feature.shape)
# Check dtype
print(feature.dtype)

(197, 61, 61, 3)
float64


In [44]:
FC[0]['Feature'].shape

(3, 61, 61)

In [45]:
len(FC)

197

In [46]:
labels = label[label['ID'].isin(labelID)]
labels = labels[['Age_1', 'TAPA', 'CVLT', 'TMT']].values
labels = labels.reshape(labels.shape[0], labels.shape[1], 1)
labels.shape

(197, 4, 1)

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test= train_test_split(feature,labels,test_size=0.25)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(147, 61, 61, 3)
(50, 61, 61, 3)
(147, 4, 1)
(50, 4, 1)


In [48]:
import tensorflow as tf
import keras
import tensorflow.keras.layers as layers
print(tf.__version__)

2.12.0


In [67]:
def get_model(width, height, depth):

    inputs = keras.Input((width, height, depth))

    x = layers.Conv3D(filters=32, kernel_size=(3,3,3), activation="relu", kernel_initializer='he_uniform')(inputs)
    x = layers.MaxPool3D(pool_size=(2,2,2))(x)
#     x = layers.BatchNormalization()(x)
    x = layers.Conv3D(filters=64, kernel_size=(3,3,3), activation="relu")(x)
    x = layers.MaxPool3D(pool_size=(2,2,2))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Conv3D(filters=128, kernel_size=(3,3,1), activation="relu")(x)
#     x = layers.MaxPool3D(pool_size=(2,2,1))(x)
#     x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
#     dense_1 = layers.Dense(units=256, activation="relu")(x)
    dense_2 = layers.Dense(units=256, activation="relu")(x)
    dense_3 = layers.Dense(units=256, activation="relu")(x)
    dense_4 = layers.Dense(units=256, activation="relu")(x)
    
#     drop_1 = layers.Dropout(0.3)(dense_1)
    drop_2 = layers.Dropout(0.3)(dense_2)
    drop_3 = layers.Dropout(0.3)(dense_3)
    drop_4 = layers.Dropout(0.3)(dense_4)

#     outputs_1 = layers.Dense(units=1, activation="relu")(drop_1)
    outputs_2 = layers.Dense(units=1, activation="relu")(drop_2)
    outputs_3 = layers.Dense(units=1, activation="relu")(drop_3)
    outputs_4 = layers.Dense(units=1, activation="relu")(drop_4)

    model = keras.Model(inputs, outputs=[outputs_2,outputs_3,outputs_4], name="3dcnn")
    return model

model = get_model(width=61, height=61, depth=3)
model.summary()

ValueError: Input 0 of layer "conv3d_36" is incompatible with the layer: expected min_ndim=5, found ndim=4. Full shape received: (None, 61, 61, 3)

In [50]:
# Y_train_2=[Y_train[:,0],Y_train[:,1],Y_train[:,2],Y_train[:,3]]
# Y_test_2=[Y_test[:,0],Y_test[:,1],Y_test[:,2],Y_test[:,3]]

Y_train_2=[Y_train[:,1],Y_train[:,2],Y_train[:,3]]
Y_test_2=[Y_test[:,1],Y_test[:,2],Y_test[:,3]]

# Y_train_2

In [64]:
# # 设置动态学习率
# initial_learning_rate = 1e-4
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate, decay_steps=30, decay_rate=0.96, staircase=True
# )
# 编译
model.compile(loss=["mae","mae","mae"], optimizer="sgd", metrics=[tf.keras.metrics.RootMeanSquaredError()])

# # 保存模型
# checkpoint_cb = keras.callbacks.ModelCheckpoint(
#     "3d_image_classification.h5", save_best_only=True
# )
# # 定义早停策略
# early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

epochs = 10000
model.fit(
    X_train,Y_train_2,batch_size=5,validation_data=(X_test,Y_test_2),
    epochs=epochs,
    shuffle=True,
    verbose=2,
#     callbacks=[checkpoint_cb, early_stopping_cb],
)

Epoch 1/10000
30/30 - 2s - loss: 0.8326 - dense_61_loss: 0.2149 - dense_62_loss: 0.2878 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2867 - dense_62_root_mean_squared_error: 0.3563 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 1.3350 - val_dense_61_loss: 0.3565 - val_dense_62_loss: 0.6516 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.4144 - val_dense_62_root_mean_squared_error: 0.7519 - val_dense_63_root_mean_squared_error: 0.3822 - 2s/epoch - 61ms/step
Epoch 2/10000
30/30 - 1s - loss: 0.8376 - dense_61_loss: 0.2186 - dense_62_loss: 0.2891 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2872 - dense_62_root_mean_squared_error: 0.3589 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.9392 - val_dense_61_loss: 0.2334 - val_dense_62_loss: 0.3789 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2954 - val_dense_62_root_mean_squared_error: 0.4871 - val_dense_63_root_mean_squared_error: 0.3822 - 512ms

30/30 - 1s - loss: 0.8258 - dense_61_loss: 0.2080 - dense_62_loss: 0.2880 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2677 - dense_62_root_mean_squared_error: 0.3575 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 1.0340 - val_dense_61_loss: 0.3736 - val_dense_62_loss: 0.3334 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.4422 - val_dense_62_root_mean_squared_error: 0.4107 - val_dense_63_root_mean_squared_error: 0.3822 - 513ms/epoch - 17ms/step
Epoch 18/10000
30/30 - 1s - loss: 0.8347 - dense_61_loss: 0.2173 - dense_62_loss: 0.2875 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2846 - dense_62_root_mean_squared_error: 0.3542 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 1.2808 - val_dense_61_loss: 0.2191 - val_dense_62_loss: 0.7228 - val_dense_63_loss: 0.3389 - val_dense_61_root_mean_squared_error: 0.2801 - val_dense_62_root_mean_squared_error: 0.8091 - val_dense_63_root_mean_squared_error: 0.3904 - 512ms/epoch - 1

Epoch 34/10000
30/30 - 0s - loss: 0.8281 - dense_61_loss: 0.2088 - dense_62_loss: 0.2895 - dense_63_loss: 0.3298 - dense_61_root_mean_squared_error: 0.2687 - dense_62_root_mean_squared_error: 0.3601 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.8965 - val_dense_61_loss: 0.2066 - val_dense_62_loss: 0.3299 - val_dense_63_loss: 0.3600 - val_dense_61_root_mean_squared_error: 0.2597 - val_dense_62_root_mean_squared_error: 0.4042 - val_dense_63_root_mean_squared_error: 0.4041 - 496ms/epoch - 17ms/step
Epoch 35/10000
30/30 - 1s - loss: 0.8327 - dense_61_loss: 0.2138 - dense_62_loss: 0.2891 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2850 - dense_62_root_mean_squared_error: 0.3626 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.8590 - val_dense_61_loss: 0.2049 - val_dense_62_loss: 0.3271 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2570 - val_dense_62_root_mean_squared_error: 0.4006 - val_dense_63_root_mean_squared_error: 0.3822 - 

30/30 - 1s - loss: 0.8238 - dense_61_loss: 0.2096 - dense_62_loss: 0.2842 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2776 - dense_62_root_mean_squared_error: 0.3584 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.9345 - val_dense_61_loss: 0.2113 - val_dense_62_loss: 0.3962 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2659 - val_dense_62_root_mean_squared_error: 0.4924 - val_dense_63_root_mean_squared_error: 0.3822 - 506ms/epoch - 17ms/step
Epoch 51/10000
30/30 - 1s - loss: 0.8212 - dense_61_loss: 0.2080 - dense_62_loss: 0.2834 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2687 - dense_62_root_mean_squared_error: 0.3512 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 2.6107 - val_dense_61_loss: 0.9250 - val_dense_62_loss: 1.3587 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.9930 - val_dense_62_root_mean_squared_error: 1.4334 - val_dense_63_root_mean_squared_error: 0.3822 - 504ms/epoch - 1

Epoch 67/10000
30/30 - 1s - loss: 0.8223 - dense_61_loss: 0.2131 - dense_62_loss: 0.2791 - dense_63_loss: 0.3301 - dense_61_root_mean_squared_error: 0.2736 - dense_62_root_mean_squared_error: 0.3475 - dense_63_root_mean_squared_error: 0.4189 - val_loss: 0.8554 - val_dense_61_loss: 0.2013 - val_dense_62_loss: 0.3271 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2553 - val_dense_62_root_mean_squared_error: 0.4006 - val_dense_63_root_mean_squared_error: 0.3822 - 510ms/epoch - 17ms/step
Epoch 68/10000
30/30 - 1s - loss: 0.8197 - dense_61_loss: 0.2105 - dense_62_loss: 0.2792 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2761 - dense_62_root_mean_squared_error: 0.3513 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.8671 - val_dense_61_loss: 0.2050 - val_dense_62_loss: 0.3352 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2548 - val_dense_62_root_mean_squared_error: 0.4076 - val_dense_63_root_mean_squared_error: 0.3822 - 

...

Epoch 340/10000
30/30 - 1s - loss: 0.7220 - dense_61_loss: 0.1648 - dense_62_loss: 0.2273 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2024 - dense_62_root_mean_squared_error: 0.2953 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.8829 - val_dense_61_loss: 0.2177 - val_dense_62_loss: 0.3364 - val_dense_63_loss: 0.3288 - val_dense_61_root_mean_squared_error: 0.2707 - val_dense_62_root_mean_squared_error: 0.4016 - val_dense_63_root_mean_squared_error: 0.3836 - 502ms/epoch - 17ms/step
Epoch 341/10000
30/30 - 1s - loss: 0.7128 - dense_61_loss: 0.1591 - dense_62_loss: 0.2238 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2020 - dense_62_root_mean_squared_error: 0.2869 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 1.2068 - val_dense_61_loss: 0.2240 - val_dense_62_loss: 0.6559 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2815 - val_dense_62_root_mean_squared_error: 0.7570 - val_dense_63_root_mean_squared_error: 0.3822 

Epoch 356/10000
30/30 - 1s - loss: 0.7150 - dense_61_loss: 0.1611 - dense_62_loss: 0.2240 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2025 - dense_62_root_mean_squared_error: 0.2860 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.8611 - val_dense_61_loss: 0.2020 - val_dense_62_loss: 0.3321 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2560 - val_dense_62_root_mean_squared_error: 0.4065 - val_dense_63_root_mean_squared_error: 0.3822 - 509ms/epoch - 17ms/step
Epoch 357/10000
30/30 - 1s - loss: 0.7145 - dense_61_loss: 0.1605 - dense_62_loss: 0.2241 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.2049 - dense_62_root_mean_squared_error: 0.2902 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.8620 - val_dense_61_loss: 0.2006 - val_dense_62_loss: 0.3345 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2559 - val_dense_62_root_mean_squared_error: 0.4054 - val_dense_63_root_mean_squared_error: 0.3822 

Epoch 372/10000
30/30 - 1s - loss: 0.7067 - dense_61_loss: 0.1582 - dense_62_loss: 0.2186 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.1962 - dense_62_root_mean_squared_error: 0.2799 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.9989 - val_dense_61_loss: 0.3016 - val_dense_62_loss: 0.3625 - val_dense_63_loss: 0.3348 - val_dense_61_root_mean_squared_error: 0.3610 - val_dense_62_root_mean_squared_error: 0.4361 - val_dense_63_root_mean_squared_error: 0.3873 - 507ms/epoch - 17ms/step
Epoch 373/10000
30/30 - 1s - loss: 0.7001 - dense_61_loss: 0.1569 - dense_62_loss: 0.2133 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.1974 - dense_62_root_mean_squared_error: 0.2759 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 1.3903 - val_dense_61_loss: 0.4276 - val_dense_62_loss: 0.6022 - val_dense_63_loss: 0.3605 - val_dense_61_root_mean_squared_error: 0.4929 - val_dense_62_root_mean_squared_error: 0.7142 - val_dense_63_root_mean_squared_error: 0.3982 

Epoch 388/10000
30/30 - 1s - loss: 0.7093 - dense_61_loss: 0.1568 - dense_62_loss: 0.2226 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.1922 - dense_62_root_mean_squared_error: 0.2790 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.8870 - val_dense_61_loss: 0.2104 - val_dense_62_loss: 0.3496 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2654 - val_dense_62_root_mean_squared_error: 0.4197 - val_dense_63_root_mean_squared_error: 0.3822 - 505ms/epoch - 17ms/step
Epoch 389/10000
30/30 - 1s - loss: 0.6928 - dense_61_loss: 0.1536 - dense_62_loss: 0.2093 - dense_63_loss: 0.3299 - dense_61_root_mean_squared_error: 0.1872 - dense_62_root_mean_squared_error: 0.2776 - dense_63_root_mean_squared_error: 0.4188 - val_loss: 0.9313 - val_dense_61_loss: 0.2018 - val_dense_62_loss: 0.4025 - val_dense_63_loss: 0.3270 - val_dense_61_root_mean_squared_error: 0.2558 - val_dense_62_root_mean_squared_error: 0.4900 - val_dense_63_root_mean_squared_error: 0.3822 

KeyboardInterrupt: 

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "1"


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
print ("-----------------------------------")
print (" TensorFlow version is ", tf.__version__)
print ("-----------------------------------")
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28, 28)), tf.keras.layers.Dense(128, activation='relu'), tf.keras.layers.Dropout(0.2), tf.keras.layers.Dense(10)])
predictions = model(x_train[:1]).numpy()
tf.nn.softmax(predictions).numpy()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
print ("-----------------------------------")
print ("*Train model")
model.fit(x_train, y_train, epochs=5)
print ("-----------------------------------")
print ("*Evaluate model")
model.evaluate(x_test, y_test, verbose=2)

-----------------------------------
 TensorFlow version is  2.12.0
-----------------------------------


2023-11-06 16:14:12.337218: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


-----------------------------------
*Train model
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2931 - accuracy: 0.9151
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1390 - accuracy: 0.9592
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1064 - accuracy: 0.9686
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0879 - accuracy: 0.9728
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0729 - accuracy: 0.9776
-----------------------------------
*Evaluate model
313/313 - 1s - loss: 0.0724 - accuracy: 0.9793 - 572ms/epoch - 2ms/step


[0.07240928709506989, 0.9793000221252441]